Profiling (timing) results for USE, ConveRT and BERT.

<a href="https://colab.research.google.com/github/PolyAI-LDN/polyai-models/blob/master/examples/profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow_gpu==1.14.0
!pip install --no-deps tensorflow_text==0.1.0
!pip install tf_sentencepiece
!pip install bert-tensorflow

     |████████████████████████████████| 377.0MB 45kB/s 
     |████████████████████████████████| 491kB 41.7MB/s 
     |████████████████████████████████| 3.2MB 35.6MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
     |████████████████████████████████| 6.2MB 7.1MB/s 
     |████████████████████████████████| 1.4MB 6.9MB/s 
     |████████████████████████████████| 71kB 4.1MB/s 


In [0]:
import tensorflow_hub as tfhub
import tensorflow as tf
import tensorflow_text  # required for tokenization ops
import tf_sentencepiece

sess = None

assert tf.__version__ == "1.14.0", (
    f"found tf version {tf.__version__}, but need 1.14.0")
if not tf.test.is_gpu_available(): 
    print("Profiling on CPU. To profile on GPU please use a GPU runtime. Edit "
    "-> Notebook settings -> Hardware accelerator")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
if not tf.test.is_gpu_available(): 
    print("Profiling on CPU. To profile on GPU please use a GPU runtime. Edit "
    "-> Notebook settings -> Hardware accelerator")

In [0]:
# Load the tensorflow hub module into a fresh session.

if sess is not None:
    sess.close()

sess = tf.InteractiveSession(graph=tf.Graph())

module = tfhub.Module("http://models.poly-ai.com/convert/v1/model.tar.gz")

text_placeholder = tf.placeholder(dtype=tf.string, shape=[None])
encoding_tensor = module(text_placeholder)
encoding_dim = int(encoding_tensor.shape[1])
print(f"ConveRT encodes text to {encoding_dim}-dimensional vectors")


def convert_encode(texts):
    """Encode the given texts to the encoding space."""
    return sess.run(encoding_tensor, feed_dict={text_placeholder: texts})

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ConveRT encodes text to 1024-dimensional vectors


In [0]:
use_module = tfhub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/1")

use_encoding_tensor = use_module(text_placeholder)
use_encoding_dim = int(use_encoding_tensor.shape[1])
print(f"USE encodes text to {use_encoding_dim}-dimensional vectors")


def use_encode(texts):
    """Encode the given texts to the encoding space."""
    return sess.run(use_encoding_tensor, feed_dict={text_placeholder: texts})

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


USE encodes text to 512-dimensional vectors


In [0]:
from bert.tokenization import FullTokenizer

uri = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"

embed_fn = tfhub.Module(uri, trainable=False)

tokenization_info = embed_fn(
            signature="tokenization_info", as_dict=True)
with tf.Session() as s:
    vocab_file, do_lower_case = s.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"]
        ])
tokenizer = FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

input_ids_placeholder = tf.placeholder(
    name="input_ids", shape=[None, None], dtype=tf.int32)
input_mask_placeholder = tf.placeholder(
    name="input_mask", shape=[None, None], dtype=tf.int32)
segment_ids = tf.zeros_like(input_ids_placeholder)
bert_inputs = dict(
    input_ids=input_ids_placeholder,
    input_mask=input_mask_placeholder,
    segment_ids=segment_ids
)

bert_encoding_tensor = embed_fn(
    inputs=bert_inputs, signature="tokens", as_dict=True)[
    "sequence_output"
]
mask = tf.expand_dims(
    tf.cast(input_mask_placeholder, dtype=tf.float32), -1)
bert_encoding_tensor = tf.reduce_sum(mask * bert_encoding_tensor, axis=1)

bert_encoding_dim = int(bert_encoding_tensor.shape[1])
print(f"BERT encodes text to {bert_encoding_dim}-dimensional vectors")

def _feed_dict(texts, max_seq_len=128):
    """Create a feed dict for feeding the texts as input.
    This uses dynamic padding so that the maximum sequence length is the
    smaller of `max_seq_len` and the longest sequence actually found in the
    batch. (The code in `bert.run_classifier` always pads up to the maximum
    even if the examples in the batch are all shorter.)
    """
    all_ids = []
    for text in texts:
        tokens = ["[CLS]"] + tokenizer.tokenize(text)

        # Possibly truncate the tokens.
        tokens = tokens[:(max_seq_len - 1)]
        tokens.append("[SEP]")
        ids = tokenizer.convert_tokens_to_ids(tokens)
        all_ids.append(ids)

    max_seq_len = max(map(len, all_ids))

    input_ids = []
    input_mask = []
    for ids in all_ids:
        mask = [1] * len(ids)

        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_len:
            ids.append(0)
            mask.append(0)

        input_ids.append(ids)
        input_mask.append(mask)

    return {input_ids_placeholder: input_ids, input_mask_placeholder: input_mask}

def bert_encode(texts):
  """Encode the given texts to the encoding space."""
  return sess.run(bert_encoding_tensor, feed_dict=_feed_dict(texts))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


BERT encodes text to 1024-dimensional vectors


In [0]:
sess.run(tf.tables_initializer())
sess.run(tf.global_variables_initializer())

In [0]:
test_sentences = """An encoder is a device, circuit, transducer, software program,
 algorithm or person that converts information from one format or code to another. 
 A compressor encodes data (e.g., audio/video/images) into a smaller form (see codec) 
 An audio encoder converts analog audio to digital audio signals
A video encoder converts analog video to digital video signals An 8b/10b encoder 
creates DC balance on a communication transmission line A simple encoder or simply 
an encoder in digital electronics is a one-hot to binary converter. A rotary encoder 
converts rotary position to an analog (e.g., analog quadrature)
or digital (e.g., digital quadrature, 32-bit parallel, or USB) electronic signal.
 That is, if there are 2n input lines, and at most only one of them will ever be 
 high, the binary code of this 'hot' line is produced on the n-bit output lines. 
 The illustrated gate level example implements the simple encoder defined by 
the truth table, but it must be understood that for all the non-explicitly defined
 input combinations. It is the reverse of a Decoder in its function.
 To sum up, encoders are great. Specially ConveRT.""".split("\n")

import time

def profile_encoder(fn):
  t0 = None
  n = 0
  for i in range(100):
    if i < 2:
      # warmup
      fn(test_sentences)
      continue
    if t0 is None:
      t0 = time.time()
    fn(test_sentences)
    n += 1
  t1 = time.time()
  print((n * len(test_sentences)) / (t1 - t0), "sentences/sec")

In [0]:
profile_encoder(use_encode)

47.07364994530965 calls/sec


In [0]:
profile_encoder(convert_encode)

38.68791757054792 calls/sec


In [0]:
profile_encoder(bert_encode)

18.960607323539552 calls/sec
